In [12]:
from playsound import playsound

In [ ]:
''' проигрывает музыку '''
playsound("C:\\Users\\HP\\Desktop\\Kursovaya\\music4kurs\\MoonSonat.mp3")

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

In [2]:
file = 'C:\\Users\\HP\\Desktop\\Kursovaya\\ABC.txt';
with open(file,'r') as f:
    text = f.read()

In [3]:
song = text.split('\n\n')
with open (file,"w") as f:
    f.write(''.join(song))

In [4]:
# набор симвлов 
vocab = set(text)

# присваевает каждому уникальному символу индекс 
char_to_index = {char_: ind for ind, char_ in enumerate (vocab)} 

''' создает многомерный массив из уникальных символов
ind_to_char = np.array(vocab) '''

# создает массив индексов 
text_as_int = np.array([char_to_index[c] for c in text])

In [5]:
# временный массив символов
a1 = []

In [6]:
# добавляем символы из словаря в список
for i in vocab:
    a1.append(i)

In [7]:
# превращаем список в массив
ind_to_char = np.array(a1)

In [8]:
# размер тренировочного набора
seq_length = 100

step = 10

# матрица  из объекта text_as_int (320x101)
sequences = np.array([text_as_int[i:i+seq_length+1] for i in range(0, len(text_as_int)-seq_length-1,step)])

#матрица из объекта  sequences без последнего элемента (320х100)
input_text = np.array([seq[:-1] for seq in sequences])

#матрица из объекта  sequences без первого жлемента (320х100)
target_text = np.array([seq[1:] for seq in sequences])

In [9]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding

In [10]:
#количество символов (55)
vocab_size = len(vocab)

In [11]:
# размер выхода 
embedding_dim = 256*2
# размерность выходного пространства
rnn_units = 1024*2

# входные данные типа int, количество 100, кортежи пусиые
x = Input(shape=(seq_length,))

# Сщставляет из х  вектор размера vocab_size, на выходе должен получится вектор размера embedding_dim, кортежи пустые 
e = Embedding(vocab_size, embedding_dim)(x)

# Слой LSTM 
l = LSTM(rnn_units, return_sequences=True)(e)

''' 
Dense: реализует операцию вывод=активация(dot(input, kernel) + bias), где
активация поэлементная функция активации,
kernel матрица весов слоя,
bias вектор смещения, сформированный слоем

Вход: размервыхода (vocab_size), функция активации (activation='softmax')

softmax - это математическая функция, которая преобразует вектор чисел в вектор вероятностей,
где вероятности каждого значения пропорциональны относительному масштабу каждого значения в векторе.

'''
d = Dense(vocab_size, activation='softmax')(l)

''' 
Model: группирует слои в объект с функциями обучения и вывода
Вход: inputs - из чего хотим получить
outputs - что хотим получить
'''
model = Model(inputs=x, outputs=d)

# печатает сводку о сети
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 100)]             0         
_________________________________________________________________
embedding (Embedding)        (None, 100, 512)          28160     
_________________________________________________________________
lstm (LSTM)                  (None, 100, 2048)         20979712  
_________________________________________________________________
dense (Dense)                (None, 100, 55)           112695    
Total params: 21,120,567
Trainable params: 21,120,567
Non-trainable params: 0
_________________________________________________________________


In [13]:
'''
Адам - это алгоритм оптимизации,который можно использоватьвместо классической процедуры стохастического градиентного спускадля
итеративного обновления весов сети на основе обучающих данных.

Стохастический градиентный спуск-оптимизационный алгоритм, отличающийся от обычного градиентного спуска тем, 
что градиент оптимизируемой функции считается на каждом шаге не как сумма градиентов от каждого элемента выборки, 
а как градиент от одного, случайно выбранного элемента.

градиентный спуск -метод нахождения локального минимума или максимума функции с помощью движения вдоль градиента.
'''
from tensorflow.keras.optimizers import Adam

In [14]:
''' 
compile - настраивает модель для обучения
Вход: optimizer=Adam() - имя оптимизатора 
loss - имя целевой функции
'''

model.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy')

In [15]:
#кличество эпох
EP=5

#количество образцов 
BS = 128

'''
fit: обучает модел для фиксированного количества эпох (итераций в наборе данных)
Вход: input_text - входные данные, 
target_text - целевые данные
batch_size - количество образцов на обновление градиентов
epochs - количество эпох
'''
hist = model.fit(input_text, target_text, batch_size=BS, epochs=EP)


Epoch 1/5
3/3 [==============================] - 54s 17s/step - loss: 4.6633
Epoch 2/5
3/3 [==============================] - 50s 16s/step - loss: 3.6434
Epoch 3/5
3/3 [==============================] - 54s 17s/step - loss: 3.3952
Epoch 4/5
3/3 [==============================] - 59s 19s/step - loss: 3.3611
Epoch 5/5
3/3 [==============================] - 61s 19s/step - loss: 3.2963


In [16]:
def generate_text(model, start_string, generation_length=100):
    input_eval = np.array([char_to_index[s] for s in start_string])
    xx = np.zeros((1, seq_length))
    xx[0,-len(input_eval):] = input_eval[:]
    text_generated = []

    model.reset_states()
    for i in range(generation_length):
        predictions = model.predict(xx)[0,-1] 
        predictions = predictions.astype(np.float64)
        predictions = predictions/np.sum(predictions)    
        predicted_id = np.argmax(np.random.multinomial(1, predictions))
        xx[0,:-1] = xx[0,1:]
        xx[0,-1] = predicted_id
        text_generated.append([ind_to_char[predicted_id]]) 
        temp = [ ''.join(x) for x in text_generated ]

    return (start_string + ''.join(temp))

In [17]:
new_song = generate_text(model, "A:", generation_length=500)

In [18]:
new_song

'A:o2fF|Mfa!-\n2!uCF!pi ,M e !u  AGw2= ,:z,!F_a 2Fez-s[r ,:2V  e,rucF|"2w2Fc .-!  V ieir]Vda- sTpa[DdoG:22c:b]d-h-a -ro|cBr  o] -:orc-tpo-:| !  Aa-daicrDS|aCa -u|io:Aw|t|2w G.2 f adr F rTe | t 2ha/ 2w: aaB nm):gF=m)2|:e|2it-2  !  [  T|!aa :-CFei222t!Anma-  ]e e-[b( a aG|o- iul m"- F|aV |,a Fi/Eh)AB2  :  Ga |-.|\n 2|w2Gcr-a n ud rF:   |feFoV AA:2|rc:2]A\na     A-ba !c|V c-c  ,  ,\n f hA M , |)-,a=a,a-(o !w2 rAiGB a r2 V a|[!! r,eteu |r    )zeto2     : rp -r2G-\nAe\nib,-r|,hei:2| aAA|),(BtAa,:2rd-Ma- r -|A'

In [ ]:
# сохраняет в текстовый файл
with open('C:\\Users\\HP\\Desktop\\Kursovaya\\new_song.txt',"w") as f:
    f.write(new_song)

In [19]:
#сохраняет в adc файл
with open('C:\\Users\\HP\\Desktop\\Kursovaya\\new_song.abc',"w") as f:
    f.write(new_song)